# Exercise Set 4
This exercise set corresponds to the set presented on GitHub as specified in Lecture 5.

## Exercise 1

In [16]:
elements = (2,123.4567,10000,12345.67)
output_string = 'file_00{:1d} :    {:.2f}, {:.2e}, {:.2e}'.format(elements[0],elements[1],elements[2],elements[3])
print(output_string)

print(f'file_00{elements[0]:1d} :    {elements[1]:.2f}, {elements[2]:.2e}, {elements[3]:.2e}')

file_002 :    123.46, 1.00e+04, 1.23e+04
file_002 :    123.46, 1.00e+04, 1.23e+04


## Exercise 2

In [20]:
def iterablestring(input_iterable):
    length = len(input_iterable)
    str1 = f'The {length:d} numbers are: '
    for i in range(length):
        if i < length - 1:
            str1 = str1 + f'{input_iterable[i]:d}, '
        else:
            str1 = str1 + f'{input_iterable[i]:d}'
    return print(str1)

iterablestring([1,3,8])

The 3 numbers are: 1, 3, 8


## Exercise 3

In [28]:
def variterstring(*nums):
    length = len(nums)
    str1 = f'The {length:d} numbers are: '
    for i in range(length):
        str1 = str1 + f'{nums[i]:d}'
        if i < length - 1:
            str1 = str1 + ', '
    return print(str1)

variterstring(1,2,5,6)

The 4 numbers are: 1, 2, 5, 6


## Exercise 4

In [31]:
date_parameters = (4,30,2017,2,27)
print(f'0{date_parameters[3]:d} {date_parameters[4]:d} {date_parameters[2]:d} 0{date_parameters[0]:d} {date_parameters[1]:d}')

02 27 2017 04 30


## Exercise 5

In [51]:
input_list = ['oranges',1.3,'lemons',1.1]
input_weights = {input_list[i]:input_list[i+1] for i in range(0,len(input_list),2)}
total_weight = input_weights['oranges'] + input_weights['lemons']
print(f'The weight of an orange is {input_weights["oranges"]:.1f} and the weight of a lemon is {input_weights["lemons"]:.1f} and the total weight is {total_weight:.1f}')

The weight of an orange is 1.3 and the weight of a lemon is 1.1 and the total weight is 2.4


## Exercise 6

In [108]:
def separator(input_iterable,width):   
    #str1 = ' ' * width
    for i in range(len(input_iterable)):
        if i == 0:
            str1 = ' '*width + f'{input_iterable[i]:d}'
        else:
            str1 = str1 + f'{input_iterable[i]:{width+1}d}'
    return print(str1)

separator([8,2,4,1,3,3],3)

8   2   4   1   3   3


## Exercise 7

In [227]:
from urllib.request import urlopen
import math
import string
import statistics

gages = ['06803495','06803486']
start_date = '2000-10-1'
end_date = '2019-9-30'
starts = start_date.split('-')
ends = end_date.split('-')
start = int(starts[0])
end = int(ends[0])
years = range(start,end+1,1)

url = 'https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb&'

for i in range(len(gages)):
    str1 = 'site_no={}&'.format(gages[i])
    url = url + str1

period = '&period=&begin_date={}&end_date={}'.format(start_date,end_date)
url = url + period

#datafile = urlopen(url)
with urlopen(url) as datafile:
    lines = datafile.readlines()

#print(lines[0:20])

pound_index = []
for i in range(len(lines)):
    lines[i] = lines[i].decode('utf-8')
    if lines[i].startswith('#'):
        pound_index.append(i)

for i in range(len(pound_index)):
    del lines[pound_index[i]]

for i in range(len(lines)):
    lines[i] = lines[i].replace('-','\t')
    lines[i] = lines[i].split('\t')

with open('runoff_data.csv','w') as csv_file:
    header = 'Organization,Site_No,Year,Month,Day,Runoff,Authorization\n'
    csv_file.write(header)
    for i in range(len(lines)):
        line_out = []
        for j in range(len(lines[i])):
            line_out.append(lines[i][j])
        if len(lines[i]) < 6:
            for m in range(len(lines[i]),6):
                line_out.append('NA')
                lines[i].append('NA')
            line_out.append('NA\n')
            lines[i].append('NA\n')
        string_line = '{},{},{},{},{},{},{}'.format(line_out[0],line_out[1],line_out[2],line_out[3],line_out[4],line_out[5],line_out[6])
        csv_file.write(string_line)

for i in range(len(gages)):
    gage = gages[i]
    csv_name = 'site{}_runoffdata.csv'.format(gage)
    with open(csv_name,'w') as csv_file:
        header = 'Month,Max_Runoff,Min_Runoff,Avg_Runoff\n'
        csv_file.write(header)
        monthly_vals = []
        for k in range(1,13,1):
            values = []
            for j in range(len(lines)):
                if lines[j][1] == gage and int(lines[j][3]) == k:
                    values.append(float(lines[j][5]))
            monthly_vals.append(values)
        for k in range(12):
            month = k+1
            maximum = max(monthly_vals[k])
            minimum = min(monthly_vals[k])
            average = statistics.mean(monthly_vals[k])
            csv_file.write('{},{:.2f},{:.2f},{:.2f}\n'.format(month,maximum,minimum,average))
    annual_vals = []
    for k in range(len(years)):
        values = []
        for j in range(len(lines)):
            if lines[j][1] == gage and lines[j][2] == str(years[k]):
                values.append(float(lines[j][5])*3600*24)
        annual_vals.append(values)
        annual_vals[k] = annual_vals[k]
    total_runoff = []
    for k in range(len(years)):
        if len(annual_vals[k]) > 0:
            year = years[k]
            total_runoff.append(sum(annual_vals[k]))
    avg_annual_runoff = (statistics.mean(total_runoff)/43560)
    # Converting cubic feet per second averaged per day as given in the metadata for the file to acre-feet per
    print(f'The annual mean runoff for site number {gage} was {avg_annual_runoff:.2f} acre-feet.' )
    


The annual mean runoff for site number 06803495 was 163168.81 acre-feet.
The annual mean runoff for site number 06803486 was 45205.67 acre-feet.
